## Regressão

A **regressão** nos permite testar uma hipótese, aqui, qual a associação entre gasto de campanha e votos. Ou seja:

$$votos_i = \alpha +\beta gastos_i$$

No modelo, "i" é a unidade de observação. O que desejamos saber é a associação entre gastos e votos de candidatos em 2020. Ou seja, a unidade de observação é o candidato. Podemos também alterar a unidade de observação para partido, por exemplo. Isso refletirá no banco de dados que usaremos para rodar a regressão.

Para este exemplo, nosso banco de dados deve te candidatos em 20200 em suas linhas. Devemos, entao, seguir os passos:

* Coletar a variável x, neste caso são os gastos;
* Coletar a variável y, neste caso são os votos;
* Montar o banco de dados que relaciona y e x;
* Rodar a regressão.

In [14]:
import pandas as pd

from statsmodels.formula.api import ols     # Pacote de análise estatística

In [41]:
prefeitos = pd.read_csv("prefeitos_2020_abridged.txt", sep=",", error_bad_lines=False)
resultados = pd.read_csv("results_2020_abridged.txt", sep=",", error_bad_lines=False)
despesas = pd.read_csv("despesas_2020_abridged.txt", sep=",",  error_bad_lines=False)

Para coletar a variável x, devemos utilizar a base de dados de despesas e somar todas as despesas para cada candidato.

In [42]:
despesas["total_candidato"] = despesas.groupby(["cpf_cand"])["valor"].transform("sum")

Vamos separar as informações que nos interessa na base de resultados. Iremos juntas as bases de dados utilizando "partido" e "muni_code".
OBS: para trabalhar com vereadores não podemos fazer desta forma, pois há mais de um vereador por partido nos municípios, seria necessário utilizar o cpf do candidato, por exemplo.

In [43]:
despesas_candidato = despesas[["partido", "muni_code", "total_candidato"]]
despesas_candidato = despesas_candi.drop_duplicates()
despesas_candidato.head()

,partido,muni_code,total_candidato
1,PP,1490,77526.70
3,MDB,1074,618942.00
5,PT,1015,4996.00
6,PT,1392,224592.08
10,PT,1090,70810.45


Vamos alterar o nome das variáveis para juntar as bases de dados e selecionar apenas o primeiro turno.

In [44]:
resultados = resultados.rename(columns={"sigla_partido_hoje" : "partido"})     # Alterar o nome da variável
resultados = resultados[resultados["num_turno"]==1]     # Selecionar 1º turno
result_desp = pd.merge(resultados, despesas_candidato,
                       how = "left",
                       on = ["muni_code", "partido"]
                      )
result_desp.head()

,ano_eleicao,num_turno,sigla_uf,muni_code,partido,total_votes_race,cand_votes,max_votes,runner_up,total_candidato
0,2020,1,AC,1007,PP,7051,2311,3386,2311.0,NaN
1,2020,1,AC,1007,PDT,7051,3386,3386,2311.0,19956.00
2,2020,1,AC,1007,PT,7051,188,3386,2311.0,3940.00
3,2020,1,AC,1007,PSB,7051,735,3386,2311.0,9978.19
4,2020,1,AC,1007,SOLIDARIEDADE,7051,431,3386,2311.0,49984.00


Vamos alterar a variável "total_candidato", fica mais intuitivo pensarmos em R$ 1000, assim, cada unidade do modelo que incluir essa variável, vai ser mil reais.
Além disso, vamos criar a variável porcentagem de votos, fica mais fácil de comparar os municípios, independente do tamanho.

In [45]:
result_desp["total_candidato_1000"] = result_desp["total_candidato"] / 1000
result_desp["percent_candidato"] = (result_desp["cand_votes"] / result_desp["total_votes_race"]) * 100

### Rodando o modelo

#### Modelo 1

In [46]:
model = ols("cand_votes ~ total_candidato", data = result_desp)     # Para rodar o modelo, do lado esquerdo é a variável dependente e do lado direito a variável independente
m1 = model.fit()     # Para coletar os resultados do modelo
m1.summary()     # Para mostrar os resultados do modelo

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             cand_votes   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                 2.733e+04
Date:                Tue, 02 Nov 2021   Prob (F-statistic):               0.00
Time:                        19:27:17   Log-Likelihood:            -1.9056e+05
No. Observations:               17167   AIC:                         3.811e+05
Df Residuals:                   17165   BIC:                         3.811e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        -455.1083    127.751     -3.562      0.000    -705.512    -204.704
total_candidato     0.0563      0.000    165.325      0.000       0.056       0.057
==============================================================================
Omnibus:                    33989.636   Durbin-Watson:                   1.440
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        316953608.622
Skew:                          15.416   Prob(JB):                         0.00
Kurtosis:                     667.951   Cond. No.                     3.92e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.92e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

OBS: nos interessa o nº de observações (17167), o Intercept coef é o $(\alpha)$ (-455.1083) e a relação entre as variáveis $(\beta)$ é o total_candidato coef (0.0563).
O std err é uma estimativa de quanto $\beta$ a varia, se dividir o coeficiente pelo valor, chegaremos ao valor t, ou seja, no exemplo, nossa estimativa está 165.325 afastado da média. O [0.025 e 0.0975] é a margem de erro.


#### Modelo 2

Calcular o mesmo modelo, mas utilizando o valor de despesas em mil reais (que criamos anteriormente).

In [47]:
model_2 = ols("cand_votes ~ total_candidato_1000", data = result_desp)
m2 = model_2.fit()
m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             cand_votes   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                 2.733e+04
Date:                Tue, 02 Nov 2021   Prob (F-statistic):               0.00
Time:                        19:27:21   Log-Likelihood:            -1.9056e+05
No. Observations:               17167   AIC:                         3.811e+05
Df Residuals:                   17165   BIC:                         3.811e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             -455.1083    127.751     -3.562      0.000    -705.512    -204.704
total_candidato_1000    56.2897      0.340    165.325      0.000      55.622      56.957
==============================================================================
Omnibus:                    33989.636   Durbin-Watson:                   1.440
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        316953608.622
Skew:                          15.416   Prob(JB):                         0.00
Kurtosis:                     667.951   Cond. No.                         392.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

$\alpha$ = - 455.1083

$\beta$ = 56.2897

t = 165.325

Como o acréscimo de mil reais foi comum para todos os candidatos, isso não afeta as variáveis, é apenas uma transformação na variável.



#### Modelo 3

Neste modelo a variável dependente é a proporção de votos para o candidato. Ao utilizar essa transformação, vamos levar em conta a quantidade de eleitores no município (é também uma variável qualitativamente diferente de cand_votos).

In [48]:
model_3 = ols("percent_candidato ~ total_candidato_1000", data = result_desp)
m3 = model_3.fit()
m3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      percent_candidato   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     8.008
Date:                Tue, 02 Nov 2021   Prob (F-statistic):            0.00466
Time:                        19:27:23   Log-Likelihood:                -78055.
No. Observations:               17167   AIC:                         1.561e+05
Df Residuals:                   17165   BIC:                         1.561e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               31.6783      0.182    173.999      0.000      31.321      32.035
total_candidato_1000     0.0014      0.000      2.830      0.005       0.000       0.002
==============================================================================
Omnibus:                      574.687   Durbin-Watson:                   1.801
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              428.880
Skew:                           0.293   Prob(JB):                     7.41e-94
Kurtosis:                       2.495   Cond. No.                         392.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

$\alpha$ = 31.6783

$\beta$ = 0.0014

t = 2.830

A partir desse modelo, temos que os candidatos que não gastam nada recebem 31,67% dos votos (na teoria).
Enquanto que, se o candidato aumentar em mil reais os gastos, ele consegue 1,4% a mais de voto.

$$ votos_i = 31.6783 + 0.0014*gastos_i $$

Assim, conseguimos saber a média de votos para quem gastou um valor x, por exemplo, 10 mil reais.



#### Modelo 4

Agora, vamos adicionar uma variável de controle, utilizaremos o total_votes_race como controle para nosso modelo.

In [49]:
model_4 = ols("cand_votes ~ total_candidato_1000 + total_votes_race", data = result_desp)
m4 = model_4.fit()
m4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             cand_votes   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                 1.695e+04
Date:                Tue, 02 Nov 2021   Prob (F-statistic):               0.00
Time:                        19:27:26   Log-Likelihood:            -1.8938e+05
No. Observations:               17167   AIC:                         3.788e+05
Df Residuals:                   17164   BIC:                         3.788e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             -754.4696    119.412     -6.318      0.000    -988.530    -520.409
total_candidato_1000    46.5528      0.372    125.091      0.000      45.823      47.282
total_votes_race         0.0390      0.001     50.307      0.000       0.038       0.041
==============================================================================
Omnibus:                    26764.030   Durbin-Watson:                   1.522
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        171852355.462
Skew:                           8.958   Prob(JB):                         0.00
Kurtosis:                     492.831   Cond. No.                     1.84e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.84e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

##### Criando variáveis dummy

In [50]:
dummies = pd.get_dummies(result_desp["sigla_uf"])     # A função get_dummies() converte variáveis categóricas em variáveis dummy.
result_desp = pd.concat([result_desp, dummies], axis = 1)     # Colocar a variável criada no data frame

dummies_p = pd.get_dummies(result_desp["partido"])
result_desp = pd.concat([result_desp, dummies_p], axis = 1)
result_desp.head()

,ano_eleicao,num_turno,sigla_uf,muni_code,partido,total_votes_race,cand_votes,max_votes,runner_up,total_candidato,...,PSOL,PSTU,PT,PTB,PTC,PV,REDE,REPUBLICANOS,SOLIDARIEDADE,UP
0,2020,1,AC,1007,PP,7051,2311,3386,2311.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2020,1,AC,1007,PDT,7051,3386,3386,2311.0,19956.00,...,0,0,0,0,0,0,0,0,0,0
2,2020,1,AC,1007,PT,7051,188,3386,2311.0,3940.00,...,0,0,1,0,0,0,0,0,0,0
3,2020,1,AC,1007,PSB,7051,735,3386,2311.0,9978.19,...,0,0,0,0,0,0,0,0,0,0
4,2020,1,AC,1007,SOLIDARIEDADE,7051,431,3386,2311.0,49984.00,...,0,0,0,0,0,0,0,0,1,0


#### Modelo 5


**Vamos controlar nosso modelo por estado.**

O coeficiente da variável dummy nos dá o valor da diferença entre as observações onde a dummy = 1 em comparação a outras observações.

Por exemplo, vamos incluir a variável dummy para o estado de São Paulo. Esperaria que, como São Paulo é um estado crucial para o cenário político nacional, o número de candidatos em cada eleição é maior. Sendo assim, quanto mais candidatos, menos votos para cada candidato

In [51]:
model_sp = ols("percent_candidato ~ total_candidato_1000 + SP", data = result_desp)
msp = model_sp.fit()
msp.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      percent_candidato   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     60.06
Date:                Tue, 02 Nov 2021   Prob (F-statistic):           1.01e-26
Time:                        19:27:30   Log-Likelihood:                -77999.
No. Observations:               17167   AIC:                         1.560e+05
Df Residuals:                   17164   BIC:                         1.560e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               32.3939      0.194    167.275      0.000      32.014      32.773
total_candidato_1000     0.0015      0.000      3.026      0.002       0.001       0.002
SP                      -5.3850      0.509    -10.586      0.000      -6.382      -4.388
==============================================================================
Omnibus:                      500.316   Durbin-Watson:                   1.812
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              399.684
Skew:                           0.295   Prob(JB):                     1.62e-87
Kurtosis:                       2.540   Cond. No.                     1.11e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.11e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Na média, os candidatos de SP recebem menos votos do que o candidato dos outros estados.



#### Modelo 6

Podemos ver isso de outra forma, utilizando somente os dados de São Paulo.

In [52]:
model_sp = ols("percent_candidato ~ total_candidato_1000", data = result_desp[result_desp.SP == 1])     # Apenas São Paulo
msp = model_sp.fit()
msp.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      percent_candidato   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.067
Date:                Tue, 02 Nov 2021   Prob (F-statistic):             0.0801
Time:                        19:32:27   Log-Likelihood:                -10474.
No. Observations:                2313   AIC:                         2.095e+04
Df Residuals:                    2311   BIC:                         2.096e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               27.0151      0.477     56.645      0.000      26.080      27.950
total_candidato_1000     0.0014      0.001      1.751      0.080      -0.000       0.003
==============================================================================
Omnibus:                      166.663   Durbin-Watson:                   1.847
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              203.846
Skew:                           0.727   Prob(JB):                     5.44e-45
Kurtosis:                       3.030   Cond. No.                         604.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Modelo 7

O que devemos esperar do PSL? Se a minha hipótese de que o PSL tem mais dinheiro, a associação entre vostos e despesas será menor para os candidatos do partido do que para a média dos outros candidatos.

In [53]:
model_psl = ols("percent_candidato ~ total_candidato_1000", data = result_desp[result_desp.PSL == 1])
mpsl = model_psl.fit()
mpsl.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      percent_candidato   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.405
Date:                Tue, 02 Nov 2021   Prob (F-statistic):              0.236
Time:                        19:37:12   Log-Likelihood:                -2733.6
No. Observations:                 626   AIC:                             5471.
Df Residuals:                     624   BIC:                             5480.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               19.5410      0.799     24.459      0.000      17.972      21.110
total_candidato_1000    -0.0018      0.002     -1.185      0.236      -0.005       0.001
==============================================================================
Omnibus:                       87.297   Durbin-Watson:                   1.761
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              121.610
Skew:                           1.052   Prob(JB):                     3.91e-27
Kurtosis:                       3.486   Cond. No.                         552.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Esta mostrando uma relação negativa entre gastos e votos! O ponto estimado é negativo, mas o intervalo de confiança inclui o zero. Ou seja, dizemos que o coeficiente aqui não se distingue de zero. Dito isso, o intervalo de confiança também não inclui o coeficiente estimado anteriormente para todos os candidatos.

Logo, se a nossa hipótese nula é "o coeficiente é diferente de zero", a nosso teste não nos deixa descartar a hipótese nula.

Se a nossa hipótese é "O coeficiente para candidatos do PSL é diferente do coeficiente de outros partidos", bem, é um pouco mais complicado do que isso (envolve um teste de diferenças entre duas estimações), mas parece que sim são diferentes, isto é o coeficiente do PSL é menor.



#### Modelo 8

A diferença entre os dois coeficientes, vamos rodar um modelo com intrações.

O coeficiente da interação deverá dar a diferença e o intervalo de confiança.

In [55]:
model_int_psl = ols("percent_candidato ~ total_candidato_1000*PSL", data = result_desp)
mpsl_int = model_int_psl.fit()
mpsl_int.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      percent_candidato   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     71.08
Date:                Tue, 02 Nov 2021   Prob (F-statistic):           1.11e-45
Time:                        19:45:44   Log-Likelihood:                -77953.
No. Observations:               17167   AIC:                         1.559e+05
Df Residuals:                   17163   BIC:                         1.559e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   32.1099      0.184    174.108      0.000      31.748      32.471
total_candidato_1000         0.0018      0.001      3.606      0.000       0.001       0.003
PSL                        -12.5689      0.967    -12.996      0.000     -14.465     -10.673
total_candidato_1000:PSL    -0.0036      0.002     -1.927      0.054      -0.007     6.1e-05
==============================================================================
Omnibus:                      505.889   Durbin-Watson:                   1.804
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              394.244
Skew:                           0.287   Prob(JB):                     2.46e-86
Kurtosis:                       2.529   Cond. No.                     2.10e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.1e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""